In [1]:
#Classification: Isotherm Data, No Data
#Import statements
#everything is the same as 3 group classification except for the labels
import pandas as pd
import os
import json
import csv
import re

#Imports from Scikit learn library
import sklearn

#Vectorize the text to frequencies
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

#splits training and testing articles
from sklearn.model_selection import train_test_split

#Algorithms used
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

#tells accuracy, f-score, precision, recall
from sklearn import metrics

#Imports from Natural Language Toolkit
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk import word_tokenize

In [2]:
doi_index = []
text = []

In [3]:
pathacs = '/Users/juliachotoo/ScrapyArticles/scrapedjson/acs2/'

for filename in os.listdir(pathacs):
    with open(pathacs + filename, 'r'):
        resultacs = json.load(open(pathacs + filename, mode='r'))
        doi_index.append(resultacs['doi'])
        text.append(resultacs['text'])

In [4]:
pathspr = '/Users/juliachotoo/ScrapyArticles/scrapedjson/spr2/'

for filename in os.listdir(pathspr):
    with open(pathspr + filename, 'r'):
        resultspr = json.load(open(pathspr + filename, mode='r'))
        doi_index.append(resultspr['doi'])
        text.append(resultspr['text'])

In [5]:
pathlabels = '/Users/juliachotoo/ScrapyArticles/simpledatalabels.csv'
labels = []

with open(pathlabels, 'r') as csvfile:
    resultlabels = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in resultlabels:
        labels.append(', '.join(row))

labels[0] = labels[0].replace('\ufeff', '')
print(labels)    


['0', '0', '0', '1', '0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0',

In [7]:
df = pd.DataFrame({'data': text, 'labels': labels})
print(df.loc[0:9])

                                                data labels
0  10.1021/ic501413r Metal–organic frameworks (MO...      0
1  10.1021/jp102881e  Top of PageAbstractIntroduc...      0
2  10.1021/acs.analchem.5b00391 Metal–organic fra...      0
3  10.1021/ct400255c Top of PageAbstractIntroduct...      1
4  10.1021/ie071645b  Top of PageAbstract1. Intro...      0
5  10.1021/ic502478u Metal–organic frameworks (MO...      0
6  10.1021/cm4034319 Top of PageAbstractIntroduct...      0
7  10.1021/acs.langmuir.5b04185 In the original a...      1
8  10.1021/ic060568u  Top of PageAbstractIntroduc...      1
9  10.1021/ie500310c Top of PageAbstractIntroduct...      0


In [8]:
#tells where the data (X) is coming from and labels (y)
#shape tells the number in each
X = df.data
y = df[df.columns[1]]
print(X.shape)
print(y.shape)

#train-test split of articles, 75% training - 25% testing
#shape shows number in each
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(313,)
(313,)
(234,)
(79,)
(234,)
(79,)


In [9]:
#stemmer puts all the words with the same stem (e.g run, runner, running) to count toward that stem's frequency
def stemming_tokenizer(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in word_tokenize(text)]

#Vectorizes text from words to numbers
#uses stemmer and removes stopwords (words that don't add to the meaning)
vect = CountVectorizer(tokenizer=stemming_tokenizer, stop_words=stopwords.words('english'))

X_train_dtm = vect.fit_transform(X_train) #fit (learning from words provided) and transform training data


X_test_dtm = vect.transform(X_test) #only transform testing data (fits words from training data that computer already knows)
X_test_dtm

<79x44773 sparse matrix of type '<class 'numpy.int64'>'
	with 64936 stored elements in Compressed Sparse Row format>

In [28]:
#I tried this but it makes the accuracy lower for all algorithms so I took it out
#It weights frequencies according to how long each article is to make the data more uniform

#tf_transformer = TfidfTransformer()
#X_train_tf = tf_transformer.fit_transform(X_train_dtm)
#X_test_tf = tf_transformer.transform(X_test_dtm)

In [10]:
#Multinomial Naive Bayes Algorithm
#the way to run each algorithm and get metrics is the same for all four
nb = MultinomialNB()

In [11]:
#fit algorithm (nb) to training data only
%time nb.fit(X_train_dtm, y_train)

CPU times: user 5.91 ms, sys: 2.67 ms, total: 8.58 ms
Wall time: 6.55 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
#Have the computer predict the labels for the testing set
y_pred_class = nb.predict(X_test_dtm)

In [13]:
#How accurate the predictions were
metrics.accuracy_score(y_test, y_pred_class)

0.87341772151898733

In [14]:
#Shows how many articles classified correctly for each group and when classified wrong which group were they
metrics.confusion_matrix(y_test, y_pred_class)

array([[62,  0],
       [10,  7]])

In [61]:
#Shows precision, recall, and f-score for each group
metrics.precision_recall_fscore_support(y_test, y_pred_class)

(array([ 0.86111111,  1.        ]),
 array([ 1.        ,  0.41176471]),
 array([ 0.92537313,  0.58333333]),
 array([62, 17]))

In [15]:
#Logistic Regression Algorithm
logreg = LogisticRegression()

In [16]:
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 106 ms, sys: 2.82 ms, total: 109 ms
Wall time: 107 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
y_pred_class2 = logreg.predict(X_test_dtm)

In [18]:
metrics.accuracy_score(y_test, y_pred_class2)

0.86075949367088611

In [19]:
metrics.confusion_matrix(y_test, y_pred_class2)

array([[59,  3],
       [ 8,  9]])

In [58]:
metrics.precision_recall_fscore_support(y_test, y_pred_class2)

(array([ 0.88059701,  0.75      ]),
 array([ 0.9516129 ,  0.52941176]),
 array([ 0.91472868,  0.62068966]),
 array([62, 17]))

In [43]:
#Support Vector Classifier Algorithm
svcm = SVC(kernel='linear')
%time svcm.fit(X_train_dtm, y_train)

CPU times: user 236 ms, sys: 3.87 ms, total: 240 ms
Wall time: 238 ms


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [44]:
y_pred_class3 = svcm.predict(X_test_dtm)

In [45]:
metrics.accuracy_score(y_test, y_pred_class3)

0.92405063291139244

In [46]:
metrics.confusion_matrix(y_test, y_pred_class3)

array([[61,  1],
       [ 5, 12]])

In [57]:
metrics.precision_recall_fscore_support(y_test, y_pred_class3)

(array([ 0.92424242,  0.92307692]),
 array([ 0.98387097,  0.70588235]),
 array([ 0.953125,  0.8     ]),
 array([62, 17]))

In [26]:
#Decision Tree with AdaBoost Algorithm
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm='SAMME', n_estimators=25)
clf.fit(X_train_dtm, y_train)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=25, random_state=None)

In [27]:
y_pred_class4 = clf.predict(X_test_dtm)

In [28]:
metrics.accuracy_score(y_test, y_pred_class4)

0.92405063291139244

In [29]:
metrics.confusion_matrix(y_test, y_pred_class4)

array([[61,  1],
       [ 5, 12]])

In [30]:
metrics.precision_recall_fscore_support(y_test, y_pred_class4)

(array([ 0.92424242,  0.92307692]),
 array([ 0.98387097,  0.70588235]),
 array([ 0.953125,  0.8     ]),
 array([62, 17]))

In [16]:
#can look at which algorithms classified incorrectly 
X_test[y_test < y_pred_class4]

112    10.1021/jp9006747   Top of PageAbstractIntrodu...
Name: data, dtype: object

In [17]:
#can see which algorithms predicted into class 1 (multicomponent)
X_test[y_pred_class4 == '1']

310    https://doi.org/10.1007/s10934-011-9494-5 1 In...
227    10.1021/jp3046356 Top of PageAbstractIntroduct...
294    https://doi.org/10.1007/s10450-013-9527-2 1 In...
236    10.1021/acs.langmuir.5b01171 Top of PageAbstra...
267    10.1021/la035556p  Top of PageAbstractIntroduc...
293    https://doi.org/10.1007/s10450-016-9793-x 1 In...
4      10.1021/ie071645b  Top of PageAbstract1. Intro...
161    10.1021/jp411536d Top of PageAbstractIntroduct...
70     10.1021/acs.inorgchem.5b00722 Top of PageAbstr...
188    10.1021/cg500269h Top of PageAbstractIntroduct...
Name: data, dtype: object